# PyCity School District Analysis

In [25]:
# Dependencies and Setup
import pandas as pd

# File to Load
school_data_to_load = "Resources/schools_data_res.csv"
student_data_to_load = "Resources/students_data_res.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Do inital Merge
merge_table = pd.merge(student_data,school_data , on = "school_name")

# District Summary

In [26]:
# Calculate the number of Schools and the number of Students
total_schools = len(school_data)
total_students = len(student_data)

In [27]:
# Calculate the Total Budget
total_budget = sum(school_data["budget"])

In [28]:
# Calculate the Average Math, Average Reading Score and the Overall Passing grade
avg_math_score = student_data["math_score"].mean()
avg_reading_score = student_data["reading_score"].mean()
avg_overall_score = (avg_math_score+avg_reading_score)/2

#Round to nearest integer for readabiliby
avg_math_score = round(avg_math_score)
avg_reading_score = round(avg_reading_score)
avg_overall_score = round(avg_overall_score)

In [29]:
# Calculate the percentage of students with a passing math score and a passing reading score (70 or greater)
students_math_pass = student_data.loc[student_data["math_score"] >= 70 ].count()
students_read_pass = student_data.loc[student_data["reading_score"] >= 70 ].count()

pct_math_pass = students_math_pass[0]/total_students*100
pct_read_pass = students_read_pass[0]/total_students*100
pct_overall_pass = (pct_math_pass+pct_read_pass)/2

#Round to 1 digit for readability
pct_math_pass = round(pct_math_pass,1)
pct_read_pass = round(pct_read_pass,1)
pct_overall_pass = round(pct_overall_pass,1)

In [30]:
#Print results to screen
district_results = pd.DataFrame({"Total Schools": [total_schools],
                                   "Total Students": [total_students],
                                   "Total Budget": [total_budget],
                                   "Average Math Score": [avg_math_score],
                                   "Average Reading Score": [avg_reading_score],
                                   "% Passing Math": [pct_math_pass],
                                   "% Passing Reading": [pct_read_pass],
                                   "% Passing Overall": [pct_overall_pass]
                                   })

district_results

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
0,15,39170,24649428,79,82,75.0,85.8,80.4


#  School Summary

In [33]:
#School Summary - Create dataframe based on student data grouped by School Name
grouped_by_school = student_data.groupby(['school_name'])

In [34]:
#Create a dataframe called school_summary with 'school_name' being the index through which all additional values can be linked
school_summary = pd.DataFrame(student_data["school_name"].value_counts())

school_summary.reset_index(inplace=True)
school_summary.columns = ["School Name", "Total Students"]
school_summary = school_summary.set_index("School Name")

In [35]:
#Calculate number of students, their average reading & math score by group (=school)
#Then append to School Summary table
total_students = grouped_by_school["Student ID"].count()
school_summary["Students Total for math"] = total_students 

avg_reading_score = grouped_by_school["reading_score"].mean()
school_summary["Average Reading Score"] = avg_reading_score 

avg_math_score = grouped_by_school["math_score"].mean()
school_summary["Average Math Score"] = avg_math_score 

In [36]:
# Create a list for those student with Math scores => 70
total_students_math_over_70 = student_data.loc[student_data["math_score"] >= 70]

# Then group by schooland append to school_summary
grouped_by_school_for_math = total_students_math_over_70.groupby(['school_name'])
number_of_math_students_70 = grouped_by_school_for_math["Student ID"].count()
school_summary["Students that pass math"] = number_of_math_students_70

# Then find the percentage that passed compared to the overall student count per school and append again
pct_passing_math_70 = number_of_math_students_70/total_students
school_summary["% Math Passed"] = round(pct_passing_math_70*100,2)

In [37]:
# Create a list for those student with Reading scores => 70
total_students_read_over_70 = student_data.loc[student_data["reading_score"] >= 70]

# Then group by school and append to school_summary
grouped_by_school_for_reading = total_students_read_over_70.groupby(["school_name"])
number_of_reading_students_70 = grouped_by_school_for_reading["Student ID"].count()
school_summary["Students that pass reading"] = number_of_reading_students_70

# Then find the percentage that passed compared to the overall student count per school and append again
pct_passing_reading_70 = number_of_reading_students_70/total_students
school_summary["% Reading Passed"] = round(pct_passing_reading_70*100,2)

In [38]:
# Lastly determine the Overall - or average - of the math and reading scores and append
pct_passing_overall = round((pct_passing_reading_70 + pct_passing_math_70)/2*100,2)
school_summary["% Overall Passed"] = pct_passing_overall

In [39]:
#Again on the 'school_name' level, create a dataframe for collecting data from school_data file
about_the_school = pd.DataFrame(school_data[["school_name","type","budget"]])
about_the_school.columns = ["School Name", "Type","Budget"]
about_the_school = about_the_school.set_index("School Name")

school_summary["Budget"] = about_the_school["Budget"]
school_summary["Type"] = about_the_school["Type"]
school_summary["Budget per Student"] = (about_the_school["Budget"]/school_summary["Total Students"])

In [40]:
#Finanlize the School Summary by selecting columns in specific order
school_final_summary = school_summary[['Type','Total Students', 'Budget', 'Budget per Student', 'Average Reading Score','Average Math Score',
         '% Math Passed', '% Reading Passed', '% Overall Passed']].sort_values(by="School Name")

# Top Performing School (by passing rate)

In [41]:
#Select TOP 5 schools based on the % Overall Passed
school_final_summary.sort_values(by="% Overall Passed",ascending=False).head(5) 

,Type,Total Students,Budget,Budget per Student,Average Reading Score,Average Math Score,% Math Passed,% Reading Passed,% Overall Passed
School Name,,,,,,,,,
Cabrera High School,Charter,1858,1081356,582.0,83.975780,83.061895,94.13,97.04,95.59
Thomas High School,Charter,1635,1043130,638.0,83.848930,83.418349,93.27,97.31,95.29
Griffin High School,Charter,1468,917500,625.0,83.816757,83.351499,93.39,97.14,95.27
Pena High School,Charter,962,585858,609.0,84.044699,83.839917,94.59,95.95,95.27
Wilson High School,Charter,2283,1319574,578.0,83.989488,83.274201,93.87,96.54,95.20


# Bottom Performing Schools (by passing rate)

In [16]:
#Select the worst 5 schools based on the % Overall Passed
school_final_summary.sort_values(by="% Overall Passed",ascending=True).head(5)

,Type,Total Students,Budget,Budget per Student,Average Reading Score,Average Math Score,% Math Passed,% Reading Passed,% Overall Passed
School Name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,80.744686,76.842711,66.37,80.22,73.29
Figueroa High School,District,2949,"$1,884,411.00",$639.00,81.158020,76.711767,65.99,80.74,73.36
Huang High School,District,2917,"$1,910,635.00",$655.00,81.182722,76.629414,65.68,81.32,73.50
Johnson High School,District,4761,"$3,094,650.00",$650.00,80.966394,77.072464,66.06,81.22,73.64
Ford High School,District,2739,"$1,763,916.00",$644.00,80.746258,77.102592,68.31,79.30,73.80


# Math Scores by Grade

In [17]:
#Create a dataframe with 'school' being a summary level again
scores_by_grade_df = pd.DataFrame(student_data["school_name"].value_counts())

In [18]:
#Per grade level, summarize the math score and count the number of students to compute an average per grade
get_9TH_math = student_data.loc[student_data["grade"] == "9th", ["school_name", "math_score"]]
get_9TH_math_gp = get_9TH_math.groupby(['school_name'])
scores_by_grade_df["9th_grades"] = get_9TH_math_gp.sum()
scores_by_grade_df["9th_students"] = get_9TH_math_gp.count()
scores_by_grade_df["9th"] = scores_by_grade_df["9th_grades"]/scores_by_grade_df["9th_students"]

get_10TH_math = student_data.loc[student_data["grade"] == "10th", ["school_name","math_score"]]
get_10TH_math_gp = get_10TH_math.groupby(['school_name'])
scores_by_grade_df["10th_grades"] = get_10TH_math_gp.sum()
scores_by_grade_df["10th_students"] = get_10TH_math_gp.count()
scores_by_grade_df["10th"] = scores_by_grade_df["10th_grades"]/scores_by_grade_df["10th_students"]

get_11TH_math = student_data.loc[student_data["grade"] == "11th", ["school_name","math_score"]]
get_11TH_math_gp = get_11TH_math.groupby(['school_name'])
scores_by_grade_df["11th_grades"] = get_11TH_math_gp.sum()
scores_by_grade_df["11th_students"] = get_11TH_math_gp.count()
scores_by_grade_df["11th"] = scores_by_grade_df["11th_grades"]/scores_by_grade_df["11th_students"]

get_12TH_math = student_data.loc[student_data["grade"] == "12th", ["school_name","math_score"]]
get_12TH_math_gp = get_12TH_math.groupby(['school_name'])
scores_by_grade_df["12th_grades"] = get_12TH_math_gp.sum()
scores_by_grade_df["12th_students"] = get_12TH_math_gp.count()
scores_by_grade_df["12th"] = scores_by_grade_df["12th_grades"]/scores_by_grade_df["12th_students"]

scores_by_grade_final = scores_by_grade_df[['9th','10th','11th','12th']].sort_index(ascending=True)
scores_by_grade_final

,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


# Reading Scores by Grade

In [19]:
#Per grade level, summarize the reading score and count the number of students to compute an average per grade
get_9TH_reading = student_data.loc[student_data["grade"] == "9th", ["school_name", "reading_score"]]
get_9TH_reading_gp = get_9TH_reading.groupby(['school_name'])
scores_by_grade_df["9th_grades"] = get_9TH_reading_gp.sum()
scores_by_grade_df["9th_students"] = get_9TH_reading_gp.count()
scores_by_grade_df["9th"] = scores_by_grade_df["9th_grades"]/scores_by_grade_df["9th_students"]

get_10TH_reading = student_data.loc[student_data["grade"] == "10th", ["school_name","reading_score"]]
get_10TH_reading_gp = get_10TH_reading.groupby(['school_name'])
scores_by_grade_df["10th_grades"] = get_10TH_reading_gp.sum()
scores_by_grade_df["10th_students"] = get_10TH_reading_gp.count()
scores_by_grade_df["10th"] = scores_by_grade_df["10th_grades"]/scores_by_grade_df["10th_students"]

get_11TH_reading = student_data.loc[student_data["grade"] == "11th", ["school_name","reading_score"]]
get_11TH_reading_gp = get_11TH_reading.groupby(['school_name'])
scores_by_grade_df["11th_grades"] = get_11TH_reading_gp.sum()
scores_by_grade_df["11th_students"] = get_11TH_reading_gp.count()
scores_by_grade_df["11th"] = scores_by_grade_df["11th_grades"]/scores_by_grade_df["11th_students"]

get_12TH_reading = student_data.loc[student_data["grade"] == "12th", ["school_name","reading_score"]]
get_12TH_reading_gp = get_12TH_reading.groupby(['school_name'])
scores_by_grade_df["12th_grades"] = get_12TH_reading_gp.sum()
scores_by_grade_df["12th_students"] = get_12TH_reading_gp.count()
scores_by_grade_df["12th"] = scores_by_grade_df["12th_grades"]/scores_by_grade_df["12th_students"]

scores_by_grade_final = scores_by_grade_df[['9th','10th','11th','12th']].sort_index(ascending=True)
scores_by_grade_final

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


# Scores by School Spending

In [20]:
#Create bins and group names
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]
size_bins = [0, 1000, 2000, 5000]
groupsize_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

#Create new dataframs based on student data to which a column for 'group name' can be added
budget_per_student = merge_table["budget"]/merge_table["size"]
merge_table["budget_per_student"] = budget_per_student

merge_table["group_name"] = pd.cut(merge_table["budget_per_student"], spending_bins, labels=group_names)
merge_table.head()
merge_table["groupsize_name"] = pd.cut(merge_table["size"], size_bins, labels=groupsize_names)
merge_table.head()


,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget,budget_per_student,group_name,groupsize_name
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635,655.0,$645-675,Large (2000-5000)
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635,655.0,$645-675,Large (2000-5000)
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635,655.0,$645-675,Large (2000-5000)
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635,655.0,$645-675,Large (2000-5000)
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635,655.0,$645-675,Large (2000-5000)


In [21]:
merge_table_read_70 = pd.DataFrame(merge_table.loc[merge_table["reading_score"] >= 70])
merge_table_read_70_renamed = merge_table_read_70.rename(columns={"reading_score":"read_70"})
# merge_table_read_70_renamed.head()

merge_table_math_70 = pd.DataFrame(merge_table.loc[merge_table["math_score"] >= 70])
merge_table_math_70_renamed = merge_table_math_70.rename(columns={"math_score":"math_70"})
# merge_table_math_70_renamed.head()

merge_table_new = pd.merge(merge_table, merge_table_math_70_renamed["math_70"], left_index=True,
                 right_index=True, how="left")
merge_table_new_new = pd.merge(merge_table_new, merge_table_read_70_renamed["read_70"], left_index=True,
                 right_index=True, how="left")
merge_table_new_new.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget,budget_per_student,group_name,groupsize_name,math_70,read_70
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635,655.0,$645-675,Large (2000-5000),79.0,NaN
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635,655.0,$645-675,Large (2000-5000),NaN,94.0
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635,655.0,$645-675,Large (2000-5000),NaN,90.0
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635,655.0,$645-675,Large (2000-5000),NaN,NaN
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635,655.0,$645-675,Large (2000-5000),84.0,97.0


#  Scores by School Spending

In [22]:
#Build the output table by Budget Spending
budget_table = merge_table_new_new.groupby("group_name")
budget_table_1 = budget_table["math_score","reading_score"].mean()

budget_table_final = pd.DataFrame(budget_table_1)

budget_table_2 = merge_table_new_new.groupby("group_name")
budget_table_2 = budget_table_2["math_70"].count()/budget_table_2["math_score"].count()*100

budget_table_final["% Passed Math"]=budget_table_2

budget_table_3 = merge_table_new_new.groupby("group_name")
budget_table_3 = budget_table_3["read_70"].count()/budget_table_3["reading_score"].count()*100

budget_table_final["% Passed Reading"]=budget_table_3

budget_table_final["% Passed Overall"] = (budget_table_final["% Passed Math"]+budget_table_final["% Passed Reading"])/2
budget_table_final

,math_score,reading_score,% Passed Math,% Passed Reading,% Passed Overall
group_name,,,,,
<$585,83.363065,83.964039,93.702889,96.686558,95.194724
$585-615,83.529196,83.838414,94.124128,95.886889,95.005509
$615-645,78.061635,81.434088,71.400428,83.614770,77.507599
$645-675,77.049297,81.005604,66.230813,81.109397,73.670105


#  Scores by School Size

In [23]:
# Build the output table by School Size
size_table = merge_table_new_new.groupby("groupsize_name")
size_table_1 = size_table["math_score","reading_score"].mean()

size_table_final = pd.DataFrame(size_table_1)

size_table_2 = merge_table_new_new.groupby("groupsize_name")
size_table_2 = size_table_2["math_70"].count()/size_table_2["math_score"].count()*100

size_table_final["% Passed Math"]=size_table_2

size_table_3 = merge_table_new_new.groupby("groupsize_name")
size_table_3 = size_table_3["read_70"].count()/size_table_3["reading_score"].count()*100

size_table_final["% Passed Reading"]=size_table_3

size_table_final["% Passed Overall"] = (size_table_final["% Passed Math"]+size_table_final["% Passed Reading"])/2
size_table_final

,math_score,reading_score,% Passed Math,% Passed Reading,% Passed Overall
groupsize_name,,,,,
Small (<1000),83.828654,83.974082,93.952484,96.040317,94.996400
Medium (1000-2000),83.372682,83.867989,93.616522,96.773058,95.194790
Large (2000-5000),77.477597,81.198674,68.652380,82.125158,75.388769


#  Scores by School Type

In [24]:
# Build the output table by School Type
district_table = merge_table_new_new.groupby("type")
district_table_1 = district_table["math_score","reading_score"].mean()
district_table_1.head()

district_table_final = pd.DataFrame(district_table_1)

district_table_2 = merge_table_new_new.groupby("type")
district_table_2 = district_table_2["math_70"].count()/district_table_2["math_score"].count()*100

district_table_final["% Passed Math"]=district_table_2

district_table_3 = merge_table_new_new.groupby("type")
district_table_3 = district_table_3["read_70"].count()/district_table_3["reading_score"].count()*100

district_table_final["% Passed Reading"]=district_table_3

district_table_final["% Passed Overall"] = (district_table_final["% Passed Math"]+district_table_final["% Passed Reading"])/2
district_table_final

,math_score,reading_score,% Passed Math,% Passed Reading,% Passed Overall
type,,,,,
Charter,83.406183,83.902821,93.701821,96.645891,95.173856
District,76.987026,80.962485,66.518387,80.905249,73.711818
